In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'cora', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: cora
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the cora graph
- Number of nodes: 2708
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 175 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.72529 Time: 0.56861
Iteration: 0002 Loss: 1.71316 Time: 0.24700
Iteration: 0003 Loss: 1.72080 Time: 0.23828
Iteration: 0004 Loss: 1.65609 Time: 0.23560
Iteration: 0005 Loss: 1.69115 Time: 0.24001
Iteration: 0006 Loss: 1.59875 Time: 0.24299
Iteration: 0007 Loss: 1.64439 Time: 0.24211
Iteration: 0008 Loss: 1.62926 Time: 0.23911
Iteration: 0009 Loss: 1.56795 Time: 0.23053
Iteration: 0010 Loss: 1.55173 Time: 0.23693
Iteration: 0011 L

Iteration: 0175 Loss: 0.59948 Time: 0.22812
Iteration: 0176 Loss: 0.59281 Time: 0.25387
Iteration: 0177 Loss: 0.59251 Time: 0.23084
Iteration: 0178 Loss: 0.58844 Time: 0.23311
Iteration: 0179 Loss: 0.58641 Time: 0.23404
Iteration: 0180 Loss: 0.58901 Time: 0.23388
Iteration: 0181 Loss: 0.58677 Time: 0.23796
Iteration: 0182 Loss: 0.58164 Time: 0.23005
Iteration: 0183 Loss: 0.58071 Time: 0.23049
Iteration: 0184 Loss: 0.58223 Time: 0.24045
Iteration: 0185 Loss: 0.58152 Time: 0.23399
Iteration: 0186 Loss: 0.58108 Time: 0.23175
Iteration: 0187 Loss: 0.58035 Time: 0.23130
Iteration: 0188 Loss: 0.57490 Time: 0.23517
Iteration: 0189 Loss: 0.57942 Time: 0.23000
Iteration: 0190 Loss: 0.57529 Time: 0.23209
Iteration: 0191 Loss: 0.57349 Time: 0.24904
Iteration: 0192 Loss: 0.57141 Time: 0.23026
Iteration: 0193 Loss: 0.57245 Time: 0.22896
Iteration: 0194 Loss: 0.57089 Time: 0.24192
Iteration: 0195 Loss: 0.57113 Time: 0.22880
Iteration: 0196 Loss: 0.57119 Time: 0.23205
Iteration: 0197 Loss: 0.57066 Ti

Iteration: 0146 Loss: 0.64796 Time: 0.23249
Iteration: 0147 Loss: 0.64524 Time: 0.23532
Iteration: 0148 Loss: 0.63940 Time: 0.23692
Iteration: 0149 Loss: 0.63937 Time: 0.22368
Iteration: 0150 Loss: 0.64346 Time: 0.23774
Iteration: 0151 Loss: 0.63764 Time: 0.23745
Iteration: 0152 Loss: 0.63636 Time: 0.23834
Iteration: 0153 Loss: 0.62850 Time: 0.23779
Iteration: 0154 Loss: 0.62983 Time: 0.23395
Iteration: 0155 Loss: 0.62752 Time: 0.22899
Iteration: 0156 Loss: 0.62009 Time: 0.23614
Iteration: 0157 Loss: 0.62015 Time: 0.23207
Iteration: 0158 Loss: 0.61901 Time: 0.23769
Iteration: 0159 Loss: 0.62227 Time: 0.22918
Iteration: 0160 Loss: 0.61906 Time: 0.23142
Iteration: 0161 Loss: 0.61212 Time: 0.23933
Iteration: 0162 Loss: 0.60972 Time: 0.23455
Iteration: 0163 Loss: 0.61150 Time: 0.23259
Iteration: 0164 Loss: 0.61259 Time: 0.23115
Iteration: 0165 Loss: 0.60565 Time: 0.23732
Iteration: 0166 Loss: 0.60694 Time: 0.22744
Iteration: 0167 Loss: 0.60136 Time: 0.23647
Iteration: 0168 Loss: 0.60712 Ti

Iteration: 0117 Loss: 0.75658 Time: 0.24301
Iteration: 0118 Loss: 0.74887 Time: 0.23641
Iteration: 0119 Loss: 0.73830 Time: 0.22597
Iteration: 0120 Loss: 0.73931 Time: 0.22894
Iteration: 0121 Loss: 0.73170 Time: 0.23999
Iteration: 0122 Loss: 0.72929 Time: 0.23378
Iteration: 0123 Loss: 0.72690 Time: 0.23200
Iteration: 0124 Loss: 0.72706 Time: 0.24317
Iteration: 0125 Loss: 0.71988 Time: 0.23416
Iteration: 0126 Loss: 0.71168 Time: 0.23020
Iteration: 0127 Loss: 0.71883 Time: 0.23735
Iteration: 0128 Loss: 0.70414 Time: 0.22933
Iteration: 0129 Loss: 0.70737 Time: 0.22942
Iteration: 0130 Loss: 0.69450 Time: 0.24700
Iteration: 0131 Loss: 0.69482 Time: 0.22701
Iteration: 0132 Loss: 0.68979 Time: 0.23819
Iteration: 0133 Loss: 0.68636 Time: 0.23500
Iteration: 0134 Loss: 0.68303 Time: 0.23201
Iteration: 0135 Loss: 0.68838 Time: 0.23806
Iteration: 0136 Loss: 0.67195 Time: 0.22977
Iteration: 0137 Loss: 0.67050 Time: 0.23546
Iteration: 0138 Loss: 0.67196 Time: 0.23155
Iteration: 0139 Loss: 0.67077 Ti

Iteration: 0088 Loss: 0.89455 Time: 0.24339
Iteration: 0089 Loss: 0.88935 Time: 0.22461
Iteration: 0090 Loss: 0.88858 Time: 0.23908
Iteration: 0091 Loss: 0.88259 Time: 0.22849
Iteration: 0092 Loss: 0.87800 Time: 0.22925
Iteration: 0093 Loss: 0.87312 Time: 0.23801
Iteration: 0094 Loss: 0.87185 Time: 0.23323
Iteration: 0095 Loss: 0.86786 Time: 0.23812
Iteration: 0096 Loss: 0.86086 Time: 0.23892
Iteration: 0097 Loss: 0.85666 Time: 0.23168
Iteration: 0098 Loss: 0.85026 Time: 0.23289
Iteration: 0099 Loss: 0.83347 Time: 0.23884
Iteration: 0100 Loss: 0.84060 Time: 0.22853
Iteration: 0101 Loss: 0.82630 Time: 0.22474
Iteration: 0102 Loss: 0.83082 Time: 0.24337
Iteration: 0103 Loss: 0.81803 Time: 0.23300
Iteration: 0104 Loss: 0.80645 Time: 0.23300
Iteration: 0105 Loss: 0.81088 Time: 0.23072
Iteration: 0106 Loss: 0.79897 Time: 0.23548
Iteration: 0107 Loss: 0.80739 Time: 0.23038
Iteration: 0108 Loss: 0.80464 Time: 0.24648
Iteration: 0109 Loss: 0.77412 Time: 0.22929
Iteration: 0110 Loss: 0.78505 Ti

...

Iteration: 0001 Loss: 1.73834 Time: 0.50231
Iteration: 0002 Loss: 1.73857 Time: 0.22973
Iteration: 0003 Loss: 1.69742 Time: 0.23987
Iteration: 0004 Loss: 1.68942 Time: 0.22825
Iteration: 0005 Loss: 1.64417 Time: 0.23610
Iteration: 0006 Loss: 1.62710 Time: 0.23918
Iteration: 0007 Loss: 1.63609 Time: 0.23110
Iteration: 0008 Loss: 1.56533 Time: 0.23494
Iteration: 0009 Loss: 1.58091 Time: 0.22900
Iteration: 0010 Loss: 1.53992 Time: 0.23143
Iteration: 0011 Loss: 1.51634 Time: 0.24861
Iteration: 0012 Loss: 1.56406 Time: 0.23783
Iteration: 0013 Loss: 1.51355 Time: 0.23202
Iteration: 0014 Loss: 1.52121 Time: 0.22583
Iteration: 0015 Loss: 1.49242 Time: 0.25303
Iteration: 0016 Loss: 1.47060 Time: 0.23199
Iteration: 0017 Loss: 1.44896 Time: 0.23230
Iteration: 0018 Loss: 1.42371 Time: 0.23585
Iteration: 0019 Loss: 1.41146 Time: 0.23616
Iteration: 0020 Loss: 1.42238 Time: 0.23163
Iteration: 0021 Loss: 1.38736 Time: 0.24504
Iteration: 0022 Loss: 1.34028 Time: 0.23163
Iteration: 0023 Loss: 1.38479 Ti

Iteration: 0188 Loss: 0.57997 Time: 0.23421
Iteration: 0189 Loss: 0.57409 Time: 0.23095
Iteration: 0190 Loss: 0.57636 Time: 0.23411
Iteration: 0191 Loss: 0.57561 Time: 0.22548
Iteration: 0192 Loss: 0.57458 Time: 0.23492
Iteration: 0193 Loss: 0.57155 Time: 0.23081
Iteration: 0194 Loss: 0.57049 Time: 0.23884
Iteration: 0195 Loss: 0.56673 Time: 0.23919
Iteration: 0196 Loss: 0.57028 Time: 0.23395
Iteration: 0197 Loss: 0.56904 Time: 0.23393
Iteration: 0198 Loss: 0.56636 Time: 0.23115
Iteration: 0199 Loss: 0.56889 Time: 0.23453
Iteration: 0200 Loss: 0.56462 Time: 0.23448
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 8 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a prep

Iteration: 0159 Loss: 0.61368 Time: 0.23128
Iteration: 0160 Loss: 0.61050 Time: 0.23119
Iteration: 0161 Loss: 0.61746 Time: 0.23758
Iteration: 0162 Loss: 0.60931 Time: 0.24099
Iteration: 0163 Loss: 0.60733 Time: 0.23600
Iteration: 0164 Loss: 0.61303 Time: 0.22609
Iteration: 0165 Loss: 0.60981 Time: 0.24840
Iteration: 0166 Loss: 0.60210 Time: 0.23582
Iteration: 0167 Loss: 0.60016 Time: 0.23249
Iteration: 0168 Loss: 0.60279 Time: 0.23691
Iteration: 0169 Loss: 0.60204 Time: 0.23421
Iteration: 0170 Loss: 0.60041 Time: 0.22519
Iteration: 0171 Loss: 0.60039 Time: 0.23605
Iteration: 0172 Loss: 0.60171 Time: 0.24196
Iteration: 0173 Loss: 0.59748 Time: 0.23599
Iteration: 0174 Loss: 0.59677 Time: 0.23334
Iteration: 0175 Loss: 0.58642 Time: 0.23290
Iteration: 0176 Loss: 0.58956 Time: 0.23499
Iteration: 0177 Loss: 0.58966 Time: 0.23133
Iteration: 0178 Loss: 0.58912 Time: 0.23399
Iteration: 0179 Loss: 0.58703 Time: 0.22617
Iteration: 0180 Loss: 0.58788 Time: 0.23478
Iteration: 0181 Loss: 0.58454 Ti

Iteration: 0130 Loss: 0.70531 Time: 0.22986
Iteration: 0131 Loss: 0.69868 Time: 0.23172
Iteration: 0132 Loss: 0.69374 Time: 0.23069
Iteration: 0133 Loss: 0.68665 Time: 0.23000
Iteration: 0134 Loss: 0.68380 Time: 0.23594
Iteration: 0135 Loss: 0.68410 Time: 0.24639
Iteration: 0136 Loss: 0.68187 Time: 0.23200
Iteration: 0137 Loss: 0.67341 Time: 0.24641
Iteration: 0138 Loss: 0.67598 Time: 0.23716
Iteration: 0139 Loss: 0.66772 Time: 0.23231
Iteration: 0140 Loss: 0.66603 Time: 0.22681
Iteration: 0141 Loss: 0.66304 Time: 0.23547
Iteration: 0142 Loss: 0.65664 Time: 0.23844
Iteration: 0143 Loss: 0.65012 Time: 0.23515
Iteration: 0144 Loss: 0.65698 Time: 0.23177
Iteration: 0145 Loss: 0.65234 Time: 0.23897
Iteration: 0146 Loss: 0.65202 Time: 0.23735
Iteration: 0147 Loss: 0.65218 Time: 0.24231
Iteration: 0148 Loss: 0.63930 Time: 0.23953
Iteration: 0149 Loss: 0.63894 Time: 0.23671
Iteration: 0150 Loss: 0.64059 Time: 0.23849
Iteration: 0151 Loss: 0.63430 Time: 0.23498
Iteration: 0152 Loss: 0.63591 Ti

Iteration: 0101 Loss: 0.85076 Time: 0.23800
Iteration: 0102 Loss: 0.82708 Time: 0.23783
Iteration: 0103 Loss: 0.83239 Time: 0.22700
Iteration: 0104 Loss: 0.82354 Time: 0.24801
Iteration: 0105 Loss: 0.81746 Time: 0.23720
Iteration: 0106 Loss: 0.82467 Time: 0.23400
Iteration: 0107 Loss: 0.81408 Time: 0.23115
Iteration: 0108 Loss: 0.80351 Time: 0.23874
Iteration: 0109 Loss: 0.79628 Time: 0.24700
Iteration: 0110 Loss: 0.79636 Time: 0.23810
Iteration: 0111 Loss: 0.78785 Time: 0.24023
Iteration: 0112 Loss: 0.79513 Time: 0.24207
Iteration: 0113 Loss: 0.77980 Time: 0.24280
Iteration: 0114 Loss: 0.77517 Time: 0.23259
Iteration: 0115 Loss: 0.77550 Time: 0.23700
Iteration: 0116 Loss: 0.76964 Time: 0.23752
Iteration: 0117 Loss: 0.76386 Time: 0.23842
Iteration: 0118 Loss: 0.76083 Time: 0.24018
Iteration: 0119 Loss: 0.75179 Time: 0.23968
Iteration: 0120 Loss: 0.75672 Time: 0.23785
Iteration: 0121 Loss: 0.73426 Time: 0.23199
Iteration: 0122 Loss: 0.74046 Time: 0.24883
Iteration: 0123 Loss: 0.73736 Ti

In [5]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on cora
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.8647361276640178
Std of AUC scores:  0.010520747321905303 

Mean AP score:  0.8935980436036692
Std of AP scores:  0.008768829726087371 
 

